In [3]:
# Data Cleaning

import pandas as pd

import urllib.parse

import re

data = pd.read_csv('search_24.csv',skipinitialspace=True)

 

term = data['term'].astype(str).str.replace('+',' ',regex=True)

term = term.map(lambda x:urllib.parse.unquote(x))

term = term.apply(lambda x: x.strip('HTTP/1.1'))

 

user = data['user_id']

 

user_term = pd.DataFrame(user)

user_term.insert(1,"term",term,True)

user_term.style.hide_index()

# print(user_term[:5].to_string(index=False))

print()

 

dic_user_term = {}

for user in user_term['user_id'].unique():

    rows = user_term.loc[user_term['user_id'] == user]

 

    if user in dic_user_term.keys():

        dic_user_term[key].append(rows['term'].to_string(index=False))

    else:

        dic_user_term[user] = rows['term'].to_string(index=False)

 

def clean_list(list_item):

                if isinstance(list_item, list):

                                for index in range(len(list_item)):

                                                if isinstance(list_item[index], list):

                                                                list_item[index] = clean_list(list_item[index])

                                                if not isinstance(list_item[index], (int, tuple, float, list)):
                                        
                                                                list_item[index] = list_item[index].strip()

                return list_item

 

word_of_user = clean_list(dic_user_term['mauryada'].splitlines())

print(word_of_user)


['iPhone', 'barclays']


In [5]:
# Data Munging

w1 = word_of_user[0]

w2 = word_of_user[1]

list_users_cosine = []

 
import numpy as np

gloveFile = "data/glove.6B.200d.txt"

def loadGloveModel(gloveFile):

    print ("Loading Glove Model")

    with open(gloveFile, encoding="utf8" ) as f:

        content = f.readlines()

    model = {}

    for line in content:

        splitLine = line.split()

        word = splitLine[0]

        embedding = np.array([float(val) for val in splitLine[1:]])

        model[word] = embedding

    # print ("Done.",len(model)," words loaded!")

    return model

 

from nltk.corpus import stopwords

def preprocess(raw_text):
    
    from spellchecker import SpellChecker
    
     # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split
    words = letters_only_text.lower().split()
    words1 = []
    spell = SpellChecker()
    misspelled = spell.unknown(words)
    
    for word in misspelled:
    # Get the one `most likely` answer
        words1.append(spell.correction(word))
    
    
    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))
    
    return cleaned_words

 

def cosine_distance_wordembedding_method(u0, u1, s1, s2):

    import scipy
#     dic = {}
    list_cos = []
   
    
    try:
        vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
        vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
       
    except KeyError:
        vector_1 = 0
        vector_2 = 0

    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    print(u0, round((1-cosine),10))
    list_cos.append(u0)
    list_cos.append(u1)
    list_cos.append(s1)
    list_cos.append(s2)
    list_cos.append(round((1-cosine),10))
    list_users_cosine.append(list_cos)
    
#     if u0 in dic.keys():
#         dic[u0].append(s2, cosine)
#     else:
#         dic[u0] = [s2, cosine]
#     print(dic)
        
     

model = loadGloveModel(gloveFile)

# cosine_distance_wordembedding_method(w1,w2)

Loading Glove Model


In [6]:
# Compare the dictionary values
for key, value in dic_user_term.items():
    print(key,":",clean_list(value.splitlines()))

mauryada : ['iPhone', 'barclays']
buttarka : ['global traders guide', 'bank litigation', 'bank compliance']
botelhfi1 : ['bayer']
marshhar : ['renishaw', 'renishaw', 'glaxo']
baterob : ['chang jian']
wuar : ['marine harvest']
ramalins : ['malaysia']
baiausti : ['auction manual']
e01933244 : ['crossover']
leibovje : ['Ruussia', 'quasi']
zhutom : ['daily credit']
mohanvis : ['auction manual']
radhakar1 : ['gold forecast']
atallahr : ['Kai Chang', 'Oils weekly', 'Oils weekly']
petrishe : ['volatility']
nestermi1 : ['global outlook uncertainty', 'time to play defence']
lewismax5 : ['kronos']
ganevae1 : ['wirecard']
mutosaki : ['on the run', 'on the run', 'on the run', 'on the run', 'on the run', 'on the run', 'DTCC', 'DTCC', 'CDX HY', 'CDX HY', 'CDX', 'CDX', 'USD MBS', 'USD MBS', 'USD MBS', 'USD MBS']
ahmedemi : ['ackermann']
sapiolud : ['abbas khan']
debernel : ['getlink']
g09636302 : ['halma plc']
kabhish1 : ['republic services']
lferrara : ['ijb']
kulkarv9 : ['stress testing bank', 'str

In [ ]:
copy_dic = dic_user_term.copy()

for key, value in dic_user_term.items():
    val1 = list(dict.fromkeys(clean_list(value.splitlines())))
    for key2, value2 in copy_dic.items():
        val2 = list(dict.fromkeys(clean_list(value2.splitlines())))
        for term in val1:
            for term2 in val2:
                cosine_distance_wordembedding_method(key,key2,term,term2)